# District {district} Widgets

In [1]:
import altair as alt
import branca
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd

from ipywidgets import *
from siuba import *

from IPython.display import Markdown, HTML

from shared_utils import styleguide, geography_utils, map_utils
from shared_utils import calitp_color_palette as cp

alt.themes.register("calitp_theme", styleguide.calitp_theme)
# enable
alt.themes.enable("calitp_theme")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0324 17:22:45.694658056    1185 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0324 17:22:48.285186198    1185 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


ThemeRegistry.enable('calitp_theme')

In [2]:
district = 7

In [3]:
df= pd.read_parquet("dla_df.parquet")

df = df>>filter(_.dist==district)

In [4]:
def make_altair_bar(df, x_col, y_col):
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(f"{x_col}:N"),
                 y=alt.Y(f"{y_col}:Q"),
                 color=alt.Color(f"{x_col}:N", 
                                 scale=alt.Scale(range=cp.CALITP_CATEGORY_BOLD_COLORS)),
                 tooltip=alt.Tooltip([x_col, y_col])
             ).properties(
                 title=f"Average Requested Funding by {x_col.title()} Dummy Variable")
             .interactive()
            )
    
    return chart

In [5]:
#GEOSPATIAL_DATA = "https://opendata.arcgis.com/datasets/a0f1da19bffe471ea89bd6a241e5604b_0.geojson"
# That URL is out of date
# Pull query from another open data source
URL = ("https://caltrans-gis.dot.ca.gov/arcgis/rest/services/"
       "CHboundary/District_Tiger_Lines/FeatureServer/0/query?"
       "where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&"
       "inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&"
       "relationParam=&outFields=DISTRICT%2C+Region&returnGeometry=true&"
       "maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&gdbVersion=&"
       "historicMoment=&returnDistinctValues=false&returnIdsOnly=false&"
       "returnCountOnly=false&returnExtentOnly=false&orderByFields=&"
       "groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&"
       "multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&"
       "returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&"
       "returnCentroid=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&"
       "datumTransformation=&f=geojson"
      )
gdf = gpd.read_file(URL)

In [6]:
def display_folium_map(gdf, plot_col):
    
    # Add additional kwargs to folium map
    colorscale = branca.colormap.StepColormap(
                    colors=["gray", "green", "navy"], 
                    vmin=gdf[plot_col].min(), vmax=gdf[plot_col].max(),
    )
    
    popup_dict = {
        "DISTRICT": "Caltrans District",
        plot_col: f"{plot_col.title()}"
    }

    m = map_utils.make_folium_choropleth_map(
        gdf,
        plot_col = plot_col,
        popup_dict = popup_dict,
        tooltip_dict = popup_dict,
        colorscale = colorscale,
        fig_width = 500, fig_height = 800,
        title="CA Map",
    )

    return m

In [7]:
def display_ipyleaflet_map(gdf, plot_col):
    
    choropleth_dict = {
        "layer_name": f"{plot_col.upper()}",
        "MIN_VALUE": gdf[plot_col].min(),
        "MAX_VALUE": gdf[plot_col].max(),
        "plot_col_name": f"{plot_col.title()}",
        "fig_width": '100%',
        "fig_height": '100%',
        "fig_min_width_px": '400px',
        "fig_min_height_px": '600px',
    }

    colorscale = branca.colormap.linear.Greens_09

    f = map_utils.make_ipyleaflet_choropleth_map(
        gdf,
        geometry_col = "DISTRICT", 
        plot_col = plot_col,
        choropleth_dict = choropleth_dict,
        colorscale = colorscale
    )
    
    return f

In [8]:
def summarize_and_plot(df, select_col, place):
    subset = df[df[select_col]==place].rename(
        columns = {
            "fed_requested": "Federal",
            "ac_requested": "Active Construction",
            "total_requested": "Total",
        }
    )
    
    prefix_count_n = subset >> count(_.prefix)

    display(Markdown(f"**Summary Statistics for {place}**"))
    display(Markdown(f"The number of obligations {place} has is {len(prefix_count_n)}."))
    
    display(Markdown(
        f"The number of prefix codes {place} uses is {subset.prefix.nunique()}."))

    pd.set_option("display.max_columns", None)

    funds = subset[['Federal','Active Construction','Total']].describe()
    display(funds.style.format(precision=2, na_rep='MISSING', thousands=","))

    # This is not fully correct, but let's get at some .interactive() parameter for bar chart,
    # rework to be long df correctly if needed
    mean_funding = geography_utils.aggregate_by_geography(
        subset,
        group_cols = ["transit"],
        mean_cols = ["Total"], 
        rename_cols=True
     )
    
    chart = make_altair_bar(
        mean_funding.assign(
            Total_mean = mean_funding.Total_mean.round(1)), 
        x_col="transit", y_col="Total_mean")
    
    display(chart)
    
    # Test folium and ipyleaflet maps
    to_plot = subset.head(12).reset_index(drop=True)
    to_plot['DISTRICT'] = to_plot.index+1
    
    plot_col = "Total"
    to_plot = gdf.merge(to_plot, on = "DISTRICT")[
        ["DISTRICT", plot_col, "geometry"]]
    
    # Maps
    display(HTML('<h2>Folium Map (with HTML formatting)</h2>'))
    

    folium_map = display_folium_map(to_plot, plot_col)
    display(folium_map)
    
    display(HTML('<h4>Ipyleaflet Map (with non-consecutive header HTML formatting)</h4>'))
    ipyleaflet_map = display_ipyleaflet_map(to_plot, plot_col)
    display(ipyleaflet_map)
    

In [9]:
# Think about how to wrap above interactive widget into a function
# Be able to select agency, county, district, etc
# https://github.com/cal-itp/data-analyses/blob/acce66e8a6f0f476ab1fd9c4e30bc78553195596/dla_functions/functions.ipynb
def interactive_widget(df, select_col):

    dropdown = widgets.Dropdown(
        description=f"{select_col.title()}",
        options=df[select_col].sort_values().unique().tolist(),
    )
    output = widgets.Output()

    display(dropdown)
    display(output)

    def on_selection(*args):
        output.clear_output()
        with output:
            summarize_and_plot(df, select_col, dropdown.value)

    dropdown.observe(on_selection, names="value")
    on_selection()

In [10]:
interactive_widget(df, "agency")

Dropdown(description='Agency', options=('Access Services', 'Acsr', 'Agoura Hills', 'Alameda Corridor Transport…

Output()

In [11]:
interactive_widget(df, "mpo")

Dropdown(description='Mpo', options=('COFCG', 'ER NONE', 'FCOG', 'KCOG', 'MCAG', 'MCTC', 'NON-MPO', 'SANDAG', …

Output()